In [164]:
import lxml
from lxml import etree
from lxml import html
import requests as re

In [165]:
raurl = 'https://www.residentadvisor.net/events/1079805'

In [166]:
response = re.get(raurl)

In [167]:
tree = lxml.html.fromstring(response.text)

In [183]:
data = []
while true:
    

ticket_classes = tree.xpath('//*[@id="tickets"]/ul/li/@class')[0]

In [181]:
print(ticket_classes)

onsale but


In [ ]:
for classes in ticket_classes:
    if 

In [116]:
for item in tree.xpath("//li"):
    if "onsale but" in item.classes:
        print("on sale")
    else:
        print("closed :(")

closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(
closed :(


In [42]:
tree.text_content

<bound method HtmlMixin.text_content of <Element p at 0x7f6da3db64a8>>

In [ ]:
print(pl).text()


In [ ]:

mypage = """
<div class="otherclass exampleclass">some</div>
<div class="otherclass">things</div>
<div class="exampleclass">are</div>
<div class="otherclass">better</div>
<div>left</div>"""

tree = html.fromstring(mypage)

for item in tree.xpath("//div"):
    if "exampleclass" in item.classes:
        print("foo")
        
        